```
AgoraService (48kHz Float32Array)
    ↓
WhisperTranscriptionService (format conversion)
    ↓
WebSocket → NestJS TranscriptionGateway
    ↓
NestJS TranscriptionService (buffering & session management)
    ↓
HTTP → Python FastAPI (model inference)
    ↓
Python returns transcription results
    ↓
NestJS aggregates and sends to frontend
    ↓
TranscriptionDisplay updates UI
```

## Architecture

```
┌─────────────────┐    HTTP/JSON    ┌──────────────────┐
│   Frontend      │ ──────────────► │  Python FastAPI  │
│   (React)       │                 │     Server       │
└─────────────────┘                 └──────────────────┘
                                              │
                                              ▼
                                    ┌──────────────────┐
                                    │  Whisper Model   │
                                    │  (Medical v2)    │
                                    └──────────────────┘
```

## **📊 Flow Diagram**

```
Frontend (React)                    NestJS Backend                    Whisper Service (Python)
     │                                    │                                │
     │ 1. POST /transcribe               │                                │
     │    {audio_data, sessionId}        │                                │
     │ ──────────────────────────────────▶│                                │
     │                                    │                                │
     │ 2. Immediate Response              │                                │
     │    {transcriptionId, status}       │                                │
     │ ◀──────────────────────────────────│                                │
     │                                    │                                │
     │                                    │ 3. Background Processing        │
     │                                    │ ──────────────────────────────▶│
     │                                    │                                │
     │                                    │ 4. Whisper Processing           │
     │                                    │    (1-3 seconds)               │
     │                                    │                                │
     │                                    │ 5. Return Transcription        │
     │                                    │ ◀──────────────────────────────│
     │                                    │                                │
     │ 6. WebSocket: transcription:status │                                │
     │ ◀──────────────────────────────────│                                │
     │                                    │                                │
     │ 7. WebSocket: transcription:result │                                │
     │ ◀──────────────────────────────────│                                │
     │                                    │                                │
     │ 8. GET /transcriptions/{id}        │                                │
     │    (optional - for history)        │                                │
     │ ──────────────────────────────────▶│                                │
     │                                    │                                │
     │ 9. Database Response               │                                │
     │ ◀──────────────────────────────────│                                │
```
